In [1]:
import json
import os
from pathlib import Path
from typing import *

import torch as t
from torch import nn
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler
from torchinfo import summary
from torchmetrics import Accuracy, F1Score, Metric
from torchvision.datasets import ImageFolder
from torchvision.models import vgg16
from torchvision.transforms import transforms, TrivialAugmentWide
from tqdm.auto import tqdm

DEVICE = "cuda" if t.cuda.is_available() else "cpu"


In [2]:
def get_subset_images(dataset, subset):
    return [dataset.imgs[i] for i in subset.indices]


def is_on_kaggle():
    """Check environment, return True when run on Kaggle. else False"""
    if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
        return True
    else:
        return False

In [3]:
DEVICE = "cuda" if t.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}\n")
status = "running on kaggle\n" if is_on_kaggle() else "running on local environment\n"
print(status)
BATCH_SIZE = 256 if is_on_kaggle() else 32
IMAGE_SIZE = (224, 224)
EPOCHS = 50 if is_on_kaggle() else 3
NUM_WORKERS = 0
TRAIN_SIZE = 0.85
LR = 0.003
DATASET_NAME = "013_balanced"  # this is the name of the dataset which is the parent of train and test directory
VERSION = DATASET_NAME[:3]
print(f"Current version: {VERSION}\n")

STORAGE = Path("STORAGE")
DST_DIR = STORAGE / VERSION
DST_DIR.mkdir(exist_ok=True, parents=True)

data_root = Path('/kaggle/input/013-balanced' if is_on_kaggle() else f'STORAGE/{VERSION}/Data')
data_path = data_root / DATASET_NAME
dataset_dir = data_path / "train"
test_dir = data_path / "test"

Using device: cuda

running on kaggle

Current version: 013



<center><h1 style=color:yellow;>UTILS</h1></center> 

In [4]:
class LossPrettifier(object):
    STYLE = {'green': '\033[32m', 'red': '\033[91m', 'bold': '\033[1m', }
    STYLE_END = '\033[0m'

    def __init__(self, show_percentage=False):

        self.show_percentage = show_percentage
        self.color_up = 'red'
        self.color_down = 'green'
        self.loss_terms = {}

    def __call__(self, epoch=None, **kwargs):

        if epoch is not None:
            print_string = f'Epoch {epoch: 5d} '
        else:
            print_string = ''

        for key, value in kwargs.items():

            pre_value = self.loss_terms.get(key, value)

            if value > pre_value:
                indicator = '▲'
                show_color = self.STYLE[self.color_up]
            elif value == pre_value:
                indicator = ''
                show_color = ''
            else:
                indicator = '▼'
                show_color = self.STYLE[self.color_down]

            if self.show_percentage:
                show_value = 0 if pre_value == 0 else (value - pre_value) / float(pre_value)
                key_string = f'| {key}: {show_color}{value:3.2f}({show_value:+3.2%}) {indicator}'
            else:
                key_string = f'| {key}: {show_color}{value:.4f} {indicator}'

            # Trim some long outputs
            key_string_part = key_string[:32]
            print_string += key_string_part + f'{self.STYLE_END}\t'

            self.loss_terms[key] = value

        print(print_string)


In [5]:
def make_weight_for_balance_classes(images, num_classes: int):
    n_images = len(images)
    count_per_class = [0] * num_classes
    for _, image_class in images:
        count_per_class[image_class] += 1
    weight_per_class = [0.] * num_classes
    for i in range(num_classes):
        weight_per_class[i] = float(n_images) / float(count_per_class[i])
    weights = [0] * n_images
    for idx, (image, image_class) in enumerate(images):
        weights[idx] = weight_per_class[image_class]
    return weights, weight_per_class



<center><h1 style=color:orange;>DATA</h1></center> 

In [6]:
train_transform = transforms.Compose(
    [transforms.Resize(size=IMAGE_SIZE), TrivialAugmentWide(num_magnitude_bins=10),
     transforms.RandomHorizontalFlip(p=0.5), transforms.ToTensor()])

dataset = ImageFolder(dataset_dir, transform=train_transform, target_transform=None)
class_names = dataset.classes
test_dataset = ImageFolder(test_dir, transform=train_transform, target_transform=None)
train_size = int(len(dataset) * TRAIN_SIZE)
validation_size = len(dataset) - train_size
train_dataset, validation_dataset = random_split(dataset, [train_size, validation_size])
print(f"Train size: {len(train_dataset)}\nValidation size: {len(validation_dataset)}")
print(f"Test size: {len(test_dataset)}\n")

Train size: 37092
Validation size: 6546
Test size: 681



In [7]:
train_imgs = get_subset_images(dataset, train_dataset)
validation_imgs = get_subset_images(dataset, validation_dataset)

train_weight, _ = make_weight_for_balance_classes(train_imgs, num_classes=len(class_names))

val_weight, _ = make_weight_for_balance_classes(validation_imgs, num_classes=len(class_names))

train_weight = t.DoubleTensor(train_weight)
val_weight = t.DoubleTensor(val_weight)

train_sampler = WeightedRandomSampler(weights=train_weight, num_samples=len(train_weight))
val_sampler = WeightedRandomSampler(weights=val_weight, num_samples=len(val_weight))

In [8]:

train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              num_workers=NUM_WORKERS,
                              sampler=train_sampler,
                              drop_last=True)

val_dataloader = DataLoader(dataset=validation_dataset,
                            batch_size=BATCH_SIZE,
                            num_workers=NUM_WORKERS,
                            sampler=val_sampler,
                            drop_last=True)

test_dataloader = DataLoader(dataset=test_dataset,
                             batch_size=BATCH_SIZE,
                             num_workers=NUM_WORKERS,
                             shuffle=False,
                             drop_last=True)

data_loader_dict = {"train": train_dataloader,
                    "val": val_dataloader,
                    "test": test_dataloader}


<center><h1 style=color:lime;>MODEL</h1></center> 

In [9]:
class AgeRangePredictor(nn.Module):
    def __init__(self,
                 hidden_units: int,
                 output_shape: int):
        super().__init__()
        self.backbone = vgg16(weights="DEFAULT")
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-1])
        backbone_output_shape = self.backbone[0][-3].out_channels
        for param in self.backbone.parameters():
            param.requires_grad = False

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=backbone_output_shape * 7 * 7, out_features=hidden_units),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=hidden_units, out_features=int(hidden_units / 2)),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=int(hidden_units / 2), out_features=output_shape)
        )

    def forward(self, x):
        return self.classifier(self.backbone(x))


model_0 = AgeRangePredictor(hidden_units=4096,
                            output_shape=len(class_names)).to(DEVICE)
model_0_loss_fn = nn.CrossEntropyLoss().to(DEVICE)
model_0_optimizer = t.optim.Adam(params=model_0.parameters(), lr=LR)

acc_fn = Accuracy(task="multiclass",
                  num_classes=len(class_names)).to(DEVICE)
f1_fn = F1Score(task="multiclass",
                num_classes=len(class_names)).to(DEVICE)

summary(model_0, input_size=(BATCH_SIZE, 3, 224, 224))

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:08<00:00, 64.5MB/s] 


Layer (type:depth-idx)                   Output Shape              Param #
AgeRangePredictor                        [256, 5]                  --
├─Sequential: 1-1                        [256, 512, 7, 7]          --
│    └─Sequential: 2-1                   [256, 512, 7, 7]          --
│    │    └─Conv2d: 3-1                  [256, 64, 224, 224]       (1,792)
│    │    └─ReLU: 3-2                    [256, 64, 224, 224]       --
│    │    └─Conv2d: 3-3                  [256, 64, 224, 224]       (36,928)
│    │    └─ReLU: 3-4                    [256, 64, 224, 224]       --
│    │    └─MaxPool2d: 3-5               [256, 64, 112, 112]       --
│    │    └─Conv2d: 3-6                  [256, 128, 112, 112]      (73,856)
│    │    └─ReLU: 3-7                    [256, 128, 112, 112]      --
│    │    └─Conv2d: 3-8                  [256, 128, 112, 112]      (147,584)
│    │    └─ReLU: 3-9                    [256, 128, 112, 112]      --
│    │    └─MaxPool2d: 3-10              [256, 128, 56, 56]  

In [10]:
class EarlyStopper:
    """
    Early stop if validation loss doesn't improve with a value of `delta_value` after a given number of epochs
    """

    def __init__(self, patience_limit: int, model: t.nn.Module, delta_value: float = 0.0, verbose=False,
                 mode: Literal['val_loss', "loss"] = "val_loss", save_dir: Optional[Path] = None):
        """
        :param patience_limit: Number of epoch to wait
        :param delta_value: A value that validation loss need to improve, if not the training session will stop
        :param verbose:
        :param mode:
        """
        if save_dir is None:
            self.save_dir = Path.cwd()
        else:
            self.save_dir = save_dir
            self.save_dir.mkdir(parents=True, exist_ok=True)

        self.patience_limit = patience_limit
        self.model = model
        self.delta_value = delta_value
        self.best_score = float("inf")
        self.verbose = verbose
        self.wait = 0
        self.mode = mode
        self.save_path = None

    def __call__(self, result_dict: Dict):
        current_score = result_dict[self.mode][-1]
        cur_epoch = len(result_dict[self.mode])
        if self.best_score is None:
            self.best_score = current_score
        elif current_score > self.best_score + self.delta_value:
            self.wait += 1
            if self.wait >= self.patience_limit:
                if self.verbose:
                    print(
                        f"Val loss did not improve from {self.best_score} in {self.wait} epochs. Early stop at epoch {cur_epoch}")
                return True
        else:  # val_loss improve
            if self.verbose:
                print(f'Val loss improve from {self.best_score + self.delta_value} → {current_score}!')

            # remove previous checkpoint if save_path is already exist
            if self.save_path is not None:
                os.remove(self.save_path)
                if self.verbose:
                    print(f"Removed previous best checkpoint at path [{self.save_path}]")

            # save best checkpoint
            if self.checkpoint(epoch=cur_epoch):
                if self.verbose:
                    print(f"Successfully saved model at [{self.save_dir}]")
            self.best_score = current_score
            self.wait = 0
            return False

    def checkpoint(self, epoch) -> bool:
        try:
            # update save path
            self.save_path = self.save_dir / f"best_model_epoch_{epoch}.pt"
            t.save(obj=self.model.state_dict(), f=self.save_path)
            return True
        except Exception as e:
            print(e)
            print("\nFailed to save best checkpoint model")
            return False


In [11]:
class Logger:
    def __init__(self, save_dir: Optional[Path] = None):
        if save_dir is None:
            self.save_dir = Path.cwd()
        else:
            self.save_dir = save_dir
            self.save_dir.mkdir(parents=True, exist_ok=True)

    def __call__(self, result_dic: Dict[str, List[float]]) -> bool:
        try:
            with open(self.save_dir / "result.json", 'w', encoding='utf-8') as f:
                json.dump(result_dic, f, ensure_ascii=False, indent=4)
            return True
        except Exception as e:
            print(e)
            return False


In [12]:
def compute_metrics(preds: int, labels: int, metrics: List[Metric], results_dict: Dict[str, float],
                    is_val: bool = False) -> Dict[str, float]:
    """
    Compute cumulative metrics given predictions and labels
    :param is_val: is validation or not, if is_val, it will add prefix "val_" to key of dict
    :param preds: predict
    :param labels: label
    :param metrics: list of metrics
    :param results_dict: Dict[str, float] Dict of metric name and metric value to be computed
    :return:
    """
    prefix = "val_" if is_val else ""
    for metric in metrics:
        metric(preds, labels)
        results_dict[prefix + metric.__class__.__name__] += metric.compute().item()
        metric.reset()
    return results_dict

In [13]:

class Trainer:
    def __init__(self, model: t.nn.Module, dataloader: Dict[str, t.utils.data.DataLoader], loss_func: t.nn.Module,
                 optimizer: t.optim.Optimizer, num_epochs: int, metrics: List[Metric], device: Any,
                 checkpoint_dir: Path = None, callbacks: List[Callable] = None):
        self.model = model
        self.dataloader = dataloader
        self.loss_func = loss_func
        self.optimizer = optimizer
        self.num_epochs = num_epochs
        self.metrics = metrics
        self.device = device
        self.__callbacks = callbacks if callbacks is not None else []
        self.results = {f"{metric.__class__.__name__}": [] for metric in self.metrics}
        self.results.update({f"val_{metric.__class__.__name__}": [] for metric in self.metrics})
        self.loss_dict = {"loss": [], "val_loss": []}
        self.reporter = LossPrettifier(show_percentage=True)
        if checkpoint_dir is None:
            self.checkpoint_dir = Path.cwd()
        else:
            self.checkpoint_dir = checkpoint_dir
            self.checkpoint_dir.mkdir(parents=True, exist_ok=True)

    def train(self):
        for epoch in tqdm(range(1, self.num_epochs + 1), desc="EPOCH"):
            self.__train_step(epoch)
            self.__validation_step(epoch)
            self.__print_loss(epoch)
            if self.__callback():
                break
            self.__save_checkpoint(epoch)
        return {**self.loss_dict, **self.results}

    def __train_step(self, epoch):
        self.model.train()
        total_train_loss = 0
        metrics_values = {metric.__class__.__name__: 0 for metric in self.metrics}
        train_bar = tqdm(self.dataloader["train"], leave=False,
                         desc=f"{' ' * (len('EPOCH') + len(str(epoch)))}TRAIN{'' * (len('VALIDATION') - len('TRAIN'))}")
        for batch, (X, y) in enumerate(train_bar):
            X, y = X.to(self.device), y.to(self.device)
            # Forward
            y_pred = self.model(X)

            # Loss & metrics
            minibatch_loss = self.loss_func(y_pred, y)
            total_train_loss += minibatch_loss
            metrics_values = compute_metrics(preds=y_pred, labels=y, results_dict=metrics_values, metrics=self.metrics)

            # gradient decent
            self.optimizer.zero_grad()
            minibatch_loss.backward()
            self.optimizer.step()

            # update bar
            train_bar.update()

        # Save Final Loss & Metrics
        total_train_loss /= len(self.dataloader["train"])
        metrics_values = {k: v / len(self.dataloader["train"]) for k, v in metrics_values.items()}
        self.loss_dict["loss"].append(total_train_loss.item())
        for k, v in metrics_values.items():
            self.results[k].append(v)

    def __validation_step(self, epoch):
        self.model.eval()
        with t.inference_mode():
            total_val_loss = 0
            val_metrics_values = {f"val_{metric.__class__.__name__}": 0 for metric in self.metrics}
            val_bar = tqdm(self.dataloader["val"], leave=False,
                           desc=f"{' ' * (len('EPOCH') + len(str(epoch)))}VALIDATION")
            for batch, (X, y) in enumerate(val_bar):
                X, y = X.to(self.device), y.to(self.device)
                # Forward
                y_pred = self.model(X)

                # Loss & Metrics
                total_val_loss += self.loss_func(y_pred, y)
                val_metrics_values = compute_metrics(preds=y_pred, labels=y, results_dict=val_metrics_values,
                                                     metrics=self.metrics, is_val=True)
                val_bar.update()

            # Save Loss & Metrics
            total_val_loss /= len(self.dataloader["val"])
            val_metrics_values = {k: v / len(self.dataloader["val"]) for k, v in val_metrics_values.items()}
            self.loss_dict["val_loss"].append(total_val_loss.item())
            for k, v in val_metrics_values.items():
                self.results[k].append(v)

    def __save_checkpoint(self, epoch):
        """Checkpoint every epoch and delete previous epoch checkpoint"""
        checkpoint_path = self.checkpoint_dir / f"epoch_{epoch - 1}.pt"
        if epoch > 1 and checkpoint_path.is_file():
            os.remove(checkpoint_path)
            print(f"Removed epoch {epoch - 1} checkpoint")

        checkpoint_path = self.checkpoint_dir / f"epoch_{epoch}.pt"
        print(f"Checkpoint Epoch at {self.checkpoint_dir}")
        t.save(obj=self.model.state_dict(), f=checkpoint_path)

    def __callback(self):
        """Execute the list off Callable and return True if it decided to stop the model"""
        stop = False
        if len(self.__callbacks) > 0:
            callback_state = []
            for func in self.__callbacks:
                result = func({**self.loss_dict, **self.results})
                callback_state.append(result)

            if all(callback_state):
                stop = True
        return stop

    def __print_loss(self, epoch):
        self.reporter(epoch=epoch, Loss=self.loss_dict["loss"][-1], Val_loss=self.loss_dict["val_loss"][-1])


In [ ]:

early_stopper = EarlyStopper(patience_limit=8,
                             verbose=True,
                             model=model_0,
                             mode="val_loss",
                             save_dir=DST_DIR / "Checkpoint")

log_json = Logger(save_dir=DST_DIR / "Output")
trainer = Trainer(
    model=model_0,
    dataloader=data_loader_dict,
    loss_func=model_0_loss_fn,
    optimizer=model_0_optimizer,
    num_epochs=EPOCHS,
    metrics=[acc_fn, f1_fn],
    device=DEVICE,
    checkpoint_dir=DST_DIR / "Checkpoint",
    callbacks=[early_stopper, log_json]
)
history = trainer.train()

print("Trained Successfully")

EPOCH:   0%|          | 0/50 [00:00<?, ?it/s]

      TRAIN:   0%|          | 0/144 [00:00<?, ?it/s]

      VALIDATION:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch     1 | Loss: 2.36(+0.00%) 	| Val_loss: 1.09(+0.00%) 	
Val loss improve from inf → 1.085726261138916!
Successfully saved model at [STORAGE/013/Checkpoint]
Checkpoint Epoch at STORAGE/013/Checkpoint


      TRAIN:   0%|          | 0/144 [00:00<?, ?it/s]

...

       TRAIN:   0%|          | 0/144 [00:00<?, ?it/s]

       VALIDATION:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch    30 | Loss: 0.95(-0.01%) ▼	| Val_loss: 1.01(+1.90%) ▲	
Removed epoch 29 checkpoint
Checkpoint Epoch at STORAGE/013/Checkpoint


       TRAIN:   0%|          | 0/144 [00:00<?, ?it/s]

       VALIDATION:   0%|          | 0/25 [00:00<?, ?it/s]